In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import random
from data.preprocessing import random_crop, prepare_for_training_data_selected_random_range, preprocessing_for_training, preprocessing_for_testing
from model.model import record_history, init_model
from data.read_data import read_setting
from data.relationship import generate_ring
from data.data_utils import load_cifar10_data, train_test_label_to_categorical

from math import floor
from sklearn.utils import shuffle
from random import randint
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

C:\Users\KevinLBJ\Anaconda3\envs\fl\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

with tf.device('/device:GPU:0'):
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)

In [3]:
# Read detailed settings from json file
detailed_setting = read_setting()

In [4]:
# Load CIFAR-10 data
train_images, train_labels, test_images, test_labels = load_cifar10_data()
# Transfer train and test label to be categorical
train_label, test_label = train_test_label_to_categorical(train_labels, test_labels)

In [5]:
def step_decay(epoch):
    epoch = _ + epo
    # initial_lrate = 1.0 # no longer needed
    drop = 0.99
    epochs_drop = 1.0
    
    lrate = 0.2 * pow(drop, floor((1+epoch)/epochs_drop))
    return lrate

In [6]:
# adjust parameters of the model
callback = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [7]:
augment = ImageDataGenerator(preprocessing_function=preprocessing_for_training)

In [8]:
training_info = detailed_setting["training_info"]
device_client_dic = generate_ring(training_info["num_device"])
alg_selected_device = [training_info["total_demand"]/training_info["num_device"] for num in range(training_info["num_device"])]

In [9]:
device_train_data = {}
device_test_data = {}

for _ in range(training_info["num_round"]):
    print("\033[1m" + "\nRound: " + str(_) + '\033[0m')
    start_with = 0
    end_with = 0

    for device in device_client_dic:
        if (_ == 0):
            #Define an estimator model and initialize every device (e.g., all devices are initialized with same parameters)
            locals()['model_{}'.format(device)] = init_model(device, device_client_dic, training_info["num_device"])

            temp_arange = []
            temp_amount = int(alg_selected_device[device]/10) + 10

            end_with += temp_amount
            end_with %= 5000
            temp_arange = [start_with, end_with]

            train_image_temp, train_label_temp = prepare_for_training_data_selected_random_range(device, temp_arange, train_images, train_labels)
            start_with = end_with
            start_with %= 5000

            device_train_data[device] = [train_image_temp, train_label_temp]

        #Local training on each device 
        for epo in range(training_info["num_local_epoch"]):
            train_image_crop = np.stack([random_crop(device_train_data[device][0][i], 24, 24) for i in range(len(device_train_data[device][0]))], axis=0)

            # Shuffle for ten times
            for random_ in range(10):
                train_new_image, train_new_label = shuffle(train_image_crop, 
                                                           device_train_data[device][1], 
                                                           random_state=randint(0, train_image_crop.shape[0]))

            history_temp = locals()['model_{}'.format(device)].weight.fit_generator(
                augment.flow(train_new_image, train_new_label, batch_size=training_info["local_batch_size"]),
                epochs=1, 
                callbacks=[callback],
                verbose=training_info["show"])

        # Update from x^(t+1/2) to x^(t+1)   (line 4)
        locals()['model_{}'.format(device)].update_parameter()

        # Count weight_q (line 5)
        locals()['model_{}'.format(device)].count_q()

    # Update all weight_hat for each device (line 6, 7)
    for device in device_client_dic:
        for dev in device_client_dic[device]:
            locals()['model_{}'.format(device)].set_client_weight(dev, locals()['model_{}'.format(dev)].weight_q)

    # Update own weight_hat for each device (line 8)
    for device in device_client_dic:
        locals()['model_{}'.format(device)].update_own_q()

    for device in device_client_dic:
        #Evaluate with new weight
        test_d = np.stack([preprocessing_for_testing(test_images[i]) for i in range(10000)], axis=0)

        test_new_image, test_new_label = shuffle(test_d, test_label, 
                                                 random_state=randint(1, train_images.shape[0]))

        history_temp = locals()['model_{}'.format(device)].weight.evaluate(test_new_image, 
                                                                           test_new_label, 
                                                                           batch_size=batch_size=training_info["center_batch_size"],
                                                                            verbose=training_info["show"])

        #Record each round accuracy and loss for every device
        record_history(locals()['model_{}'.format(device)], history_temp)
        print("Round: " + str(_) + ", Device:" + str(device) + ", " + "Result: ", str(history_temp[1]))

for device in device_client_dic:
    print("============== Device: " + str(device) + " ==============")
    print("Accuracy: " + str(locals()['model_{}'.format(device)].history['val_acc']))
    print("Loss: " + str(locals()['model_{}'.format(device)].history['val_loss']))


Round: 0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\KevinLBJ\Anaconda3\envs\fl\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-d9c71ab051fc>", line 47, in <module>
    locals()['model_{}'.format(device)].count_q()
  File "C:\Users\KevinLBJ\Gossip-Learning\model\device_model.py", line 67, in count_q
    self.weight_q.set_weights(self.weight.get_weights())
  File "C:\Users\KevinLBJ\Anaconda3\envs\fl\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py", line 1343, in set_weights
    backend.batch_set_value(weight_value_tuples)
  File "C:\Users\KevinLBJ\Anaconda3\envs\fl\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3259, in batch_set_value
    get_session().run(assign_ops, feed_dict=feed_dict)
  File "C:\Users\KevinLBJ\Anaconda3\envs\fl\lib\site-packages\tensorflow_core\python\client\session.py", line 956, in run
    run_metadata_ptr)
  File "C:\Use

TypeError: object of type 'NoneType' has no len()